In [2]:
from gensim.models.phrases import Phrases, Phraser
from gensim.models import FastText

# Data

In [4]:
import json

# read file json data_skill_tensor.json

with open('data//train/data_skill_tensor.json', 'r') as f:
    data = json.load(f)

f.close()

data_skill = [job['skills'] for job in data]

In [5]:
# load IT skill JDs
def load_data_skill(filename):
    with open(filename, 'r') as f:
        data = json.load(f)

    f.close()
    
    return list(data.values())

skill1 = load_data_skill('data//train/IT_skill_JDs_p1.json')
skill2 = load_data_skill('data/train/IT_skill_JDs_p1.json')

In [6]:
def nomalize_skill(skill):
    for i in range(len(skill)):
        skill[i] = [item.lower() for item in skill[i]]
    
    return skill

data_skill = nomalize_skill(data_skill)
skill1 = nomalize_skill(skill1)
skill2 = nomalize_skill(skill2)

In [7]:
# concate three list
total_skill_jbs = data_skill + skill1 + skill2

# Valid data and Valid Function

In [9]:
valid_data_path = 'data/valid/top_200_valid_automation.json'
with open(valid_data_path, 'r') as f:
    valid_data = json.load(f)

f.close()

In [ ]:
# valid function
def valid_skill(prediction, valid_data): # prediction: dict, valid_data: dict
    count = 0
    for key in prediction.keys():
        

# Data Agumentation

In [7]:
import random
import numpy as np

def shuffle_data(skills, discard=0):
    # Make a copy of the input list to avoid modifying the original list
    shuffled_skills = skills.copy()
    
    # Shuffle the copy of the list in place
    random.shuffle(shuffled_skills)
    
    return shuffled_skills

def cutout_data(skills, mask_size=5):

    cutout_skills = skills.copy()
    
    mask_size = min(mask_size, len(cutout_skills))
    
    cutout_index = random.sample(range(len(skills)), mask_size)
    
    for idx in cutout_index:
        cutout_skills[idx] = ''
        

    cutout_skills = [x for x in cutout_skills if x != '']
    
    return cutout_skills


In [8]:
def create_new_data(total_skills, func, num_samples=2100):

    if num_samples > len(total_skills):
        num_samples = len(total_skills) 

    skills_augmentation_collection = []
    # pick random sample in skills jbs
    random_indexs = random.sample(range(len(total_skills)), num_samples)
    for index in random_indexs:
        mask_size = 0
        if func.__name__ == 'cutout_data':
            mask_size = random.randint(0, int(len(total_skills[index])/2))

        skills_augmentation_collection.append(func(total_skills[index], mask_size))

    return skills_augmentation_collection

In [9]:
skill1_augmentation = create_new_data(skill1, cutout_data) + create_new_data(skill1, shuffle_data)
skill2_augmentation = create_new_data(skill2, cutout_data) + create_new_data(skill2, shuffle_data)

In [10]:
total_skill_jbs += skill1_augmentation + skill2_augmentation

In [11]:
total_augmentation = create_new_data(total_skill_jbs, cutout_data, len(total_skill_jbs)) + create_new_data(total_skill_jbs, shuffle_data, len(total_skill_jbs))
total_skill_jbs += total_augmentation

In [13]:
import json

# Specify the file path where you want to save the JSON data
file_path = 'fastText_newdata/jbs_skill_augmentations.json'

# Open the file for writing and save the data as JSON
with open(file_path, 'w') as json_file:
    json.dump(total_skill_jbs, json_file, indent=4)

print(f'Saved JSON data to {file_path}')


Saved JSON data to fastText_newdata/jbs_skill_augmentations.json


# Model training

In [14]:
# Define hyperparameters
vector_size = 100  # Can adjust this to your desired vector size
window_size = 5    # Context window size
min_count = 3      # Minimum word frequency
sg = 1             # Skip-gram model (1 for skip-gram, 0 for CBOW)
hs = 0
ns_exponent = 0.75
# sorted_vocab = 1
batch_words = 100
min_n = 2
max_n = 5
bucket = 2000000
shrink_windows = True
alpha = 0.1

workers = 2
epochs = 100 # increase epoch doesn't change

In [15]:
# Initialize and train the FastText model
model = FastText(
    vector_size=vector_size,
    window=window_size,
    min_count=min_count,
    sg=sg,
    hs=hs,
    alpha=alpha,
    ns_exponent=ns_exponent,
    batch_words=batch_words,
    min_n=min_n,
    max_n=max_n,
    bucket=bucket,
    workers=workers,
    epochs=epochs
)

model.build_vocab(corpus_iterable=data_skill)

model.train(
    corpus_iterable=data_skill,
    total_words = model.corpus_total_words,
    total_examples=len(data_skill),
    epochs=model.epochs,
    
)


(17575459, 28609900)

# Test model

In [6]:
vector = model.wv['html']  # get numpy vector of a word
sims = model.wv.most_similar('hive', topn=10)  # get other similar words
sims

[('sap', 0.686120867729187),
 ('plant construction', 0.680181622505188),
 ('genomics', 0.6622642278671265),
 ('snowflake', 0.6429843306541443),
 ('scrum', 0.6354222297668457),
 ('obstetrics', 0.6297516822814941),
 ('mathematics', 0.6162903308868408),
 ('business intelligence', 0.5987536311149597),
 ('splunk', 0.5981040596961975),
 ('olap', 0.5967950820922852)]

# Save model

In [16]:
save_folder = 'fastText_newdata/1/'

In [17]:
word_vectors = model.wv

vocab = list(word_vectors.index_to_key)

# Specify the TSV file name
tsv_file = save_folder + "word_vectors_fasttext.tsv"

# Open the TSV file in write mode
with open(tsv_file, "w", encoding="utf-8") as file:
    # file.write("word\t" + "\t".join([str(i) for i in range(model.vector_size)]) + "\n")

    # Write word vectors
    for word in vocab:
        vector = "\t".join(str(val) for val in word_vectors[word])
        # file.write(f"{word}\t{vector}\n")
        file.write(f"{vector}\n")

print(f"Word vectors saved to {tsv_file}")

tsv_metadata = save_folder + 'metadata_fasttext.tsv'

with open (tsv_metadata, "w", encoding='utf-8') as file:

    file.write("word")

    for word in vocab:
        file.write(f"{word}\n")

print(f"Metadata saved to {tsv_metadata}")

Word vectors saved to fastText_newdata/1/word_vectors_fasttext.tsv
Metadata saved to fastText_newdata/1/metadata_fasttext.tsv


In [18]:
model_name = save_folder + "fasttext_model"
model.save(model_name)

# End